# <center> RAG для нормативно правовых актов </center>

In [1]:
import os

import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import torch

from pymilvus import MilvusClient

from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceEndpoint

from llamaapi import LlamaAPI

from tqdm import tqdm

import json

import pickle

# os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''
token = ''

### Вспомогательные данные

In [4]:
system_prompt = """
    Ты разговариваешь только на русском языке. Ты - профессиональный юрист. За каждый правильный ответ ты будешь получать бонус.
"""

question = """ 
    Ты разговариваешь только на русском языке. Ты специалист по юристпруденции. 
    Тебе на вход приходит нормативно правовой акт. 
    Суммаризируй его, найди основную суть документа. 
    Твой ответ должен быть кратким, в ответе просто напиши саммари без лишних вводных фраз. 
    Особое внимание удели числам, процентам, датам и внесенным изменениям.
    Нормативно-правовой акт:
    {context}
"""

question_json = """ 
    Ты разговариваешь только на русском языке. Ты специалист по юристпруденции. 
    Тебе на вход приходит нормативно правовой акт. 
    На выходе ты должен отдать только JSON в следующем формате:
    
    {{'Название акта': 'Здесь помести только название нормативно правового акта', 
    'Регистрационный номер': ' Здесь помести тольео регистрационный номер НПА',
    'Дата принятия': 'Здесь Дата принятия (когда нормативный акт был официально принят или издан)',  
    'Дата вступления': 'Дата вступления в силу (по общему правилу через 10 календарных дней после официальной публикации, если в самом акте не сказано иное; есть исключения для налоговых и бюджетных нпа - они со следующего налогового периода или периода бюджетного планирования вступают в силу)',
    'Дата опубликования': 'Дата, когда НПА был опубликован в официальном источнике, чтобы  отследить его юридическую силу',
    'Орган': ' Здесь орган, издавший акт',
    'Тип документа': 'Здесь тип документа (например закон, указ, постановление, приказ)',
    'Сфера регулирования': 'Здесь Сфера регулирования: (сфера общественных отношений, на которую распространяется действие НПА, например, трудовые отношения, налогообложение, бюджетная политика)',
    'Ключнвые слова': 'Перечисли слова по которым можно найти этот документ через запятую штук 30'
    }}

    Ны выходе должен быть только json размера 7.
    Ответ должен быть обязатьльно только в формате json.
    Нормативно-правовой акт:
    {context}
"""

In [5]:
def conntect_to_llama_by_api(llama_token):
    llama = LlamaAPI(llama_token)
    return llama

def get_answer_by_api(context, llm, system_prompt, question, max_tokens=4096, temperature=0.1, quary=''):
    api_request_json = {
        'model': 'llama3.1-70b',
        'max_tokens': max_tokens,
        'temperature': temperature,
        "messages": [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': question.format(context=context, quary=quary)}
        ]
    }
    response = llm.run(api_request_json)
    return response.json()['choices'][0]['message']['content']

In [6]:
def make_summary(docs, llm, system_prompt, question, get_answer_func=get_answer_by_api):
    summarys = []
    for doc in tqdm(docs):
        summarys.append(get_answer_func(doc, llm, system_prompt, question))
    return summarys

### Подготовка данных для хранилища

In [7]:
data_path = 'hmao_npa.txt'  # path to dataset

with open(data_path, encoding='utf8') as file:
    txt = file.read()

list_docs = txt.split('\n')
list_docs = [doc for doc in list_docs if doc]

In [8]:
new_docs = []
numbers = []
for i in range(len(list_docs)):
    list_docs[i] = list_docs[i].replace('¦', '|')
    ids = list_docs[i].find('|')

    if ids == -1:
        new_docs.append(list_docs[i])
    else:
        new_docs.append(list_docs[i][:ids])
        numbers.append(i)

### Попытка собрать метаданные

In [10]:
llama = conntect_to_llama_by_api(token)

response = get_answer_by_api(list_docs[0][:10000], llama, system_prompt, question_json)
print(response)
json_response = json.loads(response)

{
"Название акта": "ПОСТАНОВЛЕНИЕ ГУБЕРНАТОРА ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГРЫ от 28.12.2017 № 139",
"Регистрационный номер": "139",
"Дата принятия": "28 декабря 2017 года",
"Дата вступления": "8 января 2018 года",
"Дата опубликования": "28 декабря 2017 года",
"Орган": "Губернатор Ханты-Мансийского автономного округа - Югры",
"Тип документа": "Постановление",
"Сфера регулирования": "Делопроизводство в государственных органах Ханты-Мансийского автономного округа - Югры и исполнительных органах государственной власти Ханты-Мансийского автономного округа - Югры",
"Ключевые слова": "Делопроизводство, государственные органы, исполнительные органы, Ханты-Мансийский автономный округ - Югра, постановление, губернатор, нормативные правовые акты"
}


In [ ]:
metadata = []
for doc in tqdm(new_docs):
    token = ''  # llamaapi token
    llama = conntect_to_llama_by_api(token)

    while True:
        try:
            response = get_answer_by_api(doc[:10000], llama, system_prompt, question_json)
            json_response = json.loads(response)
            break
        except:
            continue
    metadata.append(json_response)
    with open('metadata.pkl', 'wb') as file:
        pickle.dump(metadata, file)

  1%|▊                                                                            | 21/1856 [19:39<21:42:17, 42.58s/it]

## Попытка сделать суммаризацию для всех документов

In [ ]:
all_summarys = []
for act in tqdm(new_docs):
    llama = conntect_to_llama_by_api(token)
    act_len = len(act)
    if act_len > 35000:
        i = 0
        summarys = []
        while act_len > i:
            end = i + 35000
            if end > act_len:
                end = act_len
            while True:
                try:
                    response_2 = get_answer_by_api(act[i:end], llama, system_prompt, question)
                    break
                except:
                    llama = conntect_to_llama_by_api(token)
                    continue
            summarys.append(response_2)
            i += 35000
        response_2 = get_answer_by_api(' '.join(summarys), llama, system_prompt, question)
        all_summarys.append(response_2)
    else:
        while True:
            try:
                response_2 = get_answer_by_api(act, llama, system_prompt, question)
                break
            except:
                llama = conntect_to_llama_by_api(token)
                continue
        all_summarys.append(response_2)
    with open('summary.pkl', 'wb') as file:
        pickle.dump(all_summarys, file)

  1%|▌                                                                            | 13/1856 [10:18<23:56:42, 46.77s/it]

In [ ]:
i = 0
summarys = []
while len(new_docs[8]) > i:
    end = i + 40000
    if end > len(new_docs[8]):
        end = len(new_docs[8])
    response_2 = get_answer_by_api(new_docs[8][i: end], llama, system_prompt, question)
    summarys.append(response_2)
    i += 40000
print(len(summarys))

In [72]:
max_len = max([len(doc.split()) for doc in new_docs])
for doc in new_docs:
    if len(doc.split()) == max_len:
        bad_doc = doc

In [63]:
count = 0
for doc in new_docs:
    if len(doc) > 40000:
        count += 1
print(count)

86


In [32]:
len(bad_doc)

186347

In [78]:
# response = get_answer_by_api(bad_doc[len(bad_doc) // 2 : ], llama, system_prompt, question_json)
# print(response)
# json_response = json.loads(response)

# print()
response_2 = get_answer_by_api(bad_doc[:40000], llama, system_prompt, question)

print(response_2)

Постановление Губернатора Ханты-Мансийского автономного округа - Югры от 24 июня 2013 года N 84 "О схеме размещения, использования и охраны охотничьих угодий на территории Ханты-Мансийского автономного округа - Югры".

Основная суть документа: утверждение схемы размещения, использования и охраны охотничьих угодий на территории Ханты-Мансийского автономного округа - Югры.

Ключевые положения:

* Схема разработана в соответствии с Федеральным законом от 24.07.2009 N 209-ФЗ "Об охоте и о сохранении охотничьих ресурсов и о внесении изменений в отдельные законодательные акты Российской Федерации" и другими нормативно-правовыми актами.
* Целью формирования схемы является планирование в области охоты и сохранения охотничьих ресурсов.
* Схема включает в себя физико-географическое описание территории, социально-экономическую характеристику, характеристику размещения и состояния использования охотничьих угодий и охотничьих ресурсов, мероприятия по организации рационального использования охотничь

In [97]:
i = 0
summarys = []
while len(new_docs[8]) > i:
    end = i + 40000
    if end > len(new_docs[8]):
        end = len(new_docs[8])
    response_2 = get_answer_by_api(new_docs[8][i: end], llama, system_prompt, question)
    summarys.append(response_2)
    i += 40000
print(len(summarys))

1


In [96]:
get_answer_by_api(' '.join(summarys), llama, system_prompt, question)

'Пожалуйста, предоставьте текст нормативно-правового акта, и я суммирую его для вас.'

In [127]:
while True:
    try:
         # Если модель уже возвращает что-то похожее на JSON
        response = get_answer_by_api(list_docs[0], llama, system_prompt, question_json)
        print(response)
        json_response = json.loads(response)
        break
    except:
        print('Eror')
        print()
        continue

{
    "Название акта": "ПОСТАНОВЛЕНИЕ ГУБЕРНАТОРА ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГРЫ",
    "Регистрационный номер": "139",
    "Дата принятия": "28.12.2017",
    "Дата вступления": "01.07.2018",
    "Дата опубликования": "Не указана",
    "Орган": "Губернатор Ханты-Мансийского автономного округа - Югры",
    "Тип документа": "Постановление",
    "Сфера регулирования": "Делопроизводство в государственных органах",
    "Ключевые слова": "Делопроизводство, государственные органы, Ханты-Мансийский автономный округ - Югра"
}


In [128]:
llama = conntect_to_llama_by_api(token)
get_answer_by_api(list_docs[0], llama, system_prompt, question_json)

'{\n  "Название акта": "ПОСТАНОВЛЕНИЕ ГУБЕРНАТОРА ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГРЫ",\n  "Регистрационный номер": "139",\n  "Дата принятия": "28.12.2017",\n  "Дата вступления": "01.07.2018",\n  "Дата опубликования": "Не указана",\n  "Орган": "Губернатор Ханты-Мансийского автономного округа - Югры",\n  "Тип документа": "Постановление",\n  "Сфера регулирования": "Делопроизводство в государственных органах",\n  "Ключевые слова": "Делопроизводство, государственные органы, Ханты-Мансийский автономный округ - Югра"\n}'

In [12]:
summarys = make_summary(list_docs[:10], llama, system_prompt, question)  # TODO: use full data

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:31<00:00,  3.18s/it]
